In [ ]:
import json

# Load the uploaded dataset
input_path = r"C:\Users\darsh\Desktop\Main Documents Folder\NewsVerse\Dataset sample\Final_smple_dataset.json"
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Remove score-related fields
score_fields = ["score"]

for item in data:
    for field in score_fields:
        if field in item:
            del item[field]

# Save cleaned dataset
output_path = r"C:\Users\darsh\Desktop\Main Documents Folder\NewsVerse\Dataset sample\Final_smple_dataset.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

output_path

'C:\\Users\\darsh\\Desktop\\Main Documents Folder\\NewsVerse\\Dataset sample\\Final_smple_dataset.json'